In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [29]:
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

## SDMTS

In [3]:
mts_itp_id = 278

In [4]:
analysis_date = dt.date(2022, 2, 17)

In [5]:
def get_vehicle_positions(itp_id, analysis_date):
    ''' 
    itp_id: an itp_id (string or integer)
    analysis_date: datetime.date
    
    Interim function for getting complete vehicle positions data for a single operator on a single date of interest.
    To be replaced as RT views are implemented...
    
    Currently drops positions for day after analysis date after 2AM, temporary fix to balance capturing trips crossing
    midnight with avoiding duplicates...
    '''

    
    next_date = analysis_date + dt.timedelta(days = 1)
    date_str = analysis_date.strftime('%Y-%m-%d')
    
    start = dt.datetime.combine(analysis_date, dt.time(0))
    start_ts = int(start.timestamp())
    end = start + dt.timedelta(days = 1, seconds = 2 * 60**2)
    end_ts = int(end.timestamp())
    
    filename = f'vp_{itp_id}_{date_str}.parquet'
    path = check_cached(filename)
    if path:
        print('found parquet')
        return pd.read_parquet(path)
    else:
        df = query_sql(f"""
        SELECT calitp_itp_id, calitp_url_number,
        header.timestamp AS header_timestamp, vehicle.timestamp AS vehicle_timestamp,
        vehicle.vehicle.label AS entity_id, vehicle.vehicle.id AS vehicle_id,
        vehicle.trip.tripId AS trip_id, vehicle.position.longitude AS vehicle_longitude,
        vehicle.position.latitude AS vehicle_latitude
        FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
        WHERE calitp_itp_id = {itp_id} AND vehicle.timestamp > {start_ts} AND vehicle.timestamp < {end_ts}
        """)
        
        df = df >> distinct(_.vehicle_trip_id, _.vehicle_timestamp, _keep_all=True)
        df = df.dropna(subset=['vehicle_timestamp'])
        assert not df.empty, f'no vehicle positions data found for {date_str}'
        df.vehicle_timestamp = df.vehicle_timestamp.apply(convert_ts)
        df.header_timestamp = df.header_timestamp.apply(convert_ts)

        # assert df.vehicle_timestamp.min() < dt.datetime.combine(analysis_date, dt.time(0)), 'rt data starts after analysis date'
        # assert dt.datetime.combine(analysis_date, dt.time(hour=23, minute=59)) < df.vehicle_timestamp.max(), 'rt data ends early on analysis date'
        # if not df.vehicle_timestamp.min() < dt.datetime.combine(analysis_date, dt.time(0)):
        #     warnings.warn('rt data starts after analysis date')
        # if not dt.datetime.combine(end) < df.vehicle_timestamp.max():
        #     warnings.warn('rt data ends early on analysis date')

        df.to_parquet(f'{GCS_FILE_PATH}cached_views/{filename}')
        return df

In [7]:
mts = rt.OperatorDayAnalysis(mts_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found_parquet
3 scheduled trips out of 8104 have no shape, dropping
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data


/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)


In [85]:
trolley_routes = ['510', '520', '530']

In [86]:
mts_bus = [route for route in list(
                   mts.rt_trips.route_id.unique()) if route not in trolley_routes
                         ]

In [87]:
mts.reset_filter()

In [88]:
rt.OperatorDayAnalysis.set_filter(mts, start_time='15:00', end_time='19:00')

{'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}


In [34]:
pbar = tqdm()

0it [00:00, ?it/s]

In [89]:
m = mts.segment_speed_map(size = [1300, 700])

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}
negative speed for shape 705_2_35, dropping
negative speed for shape 992_9_52, dropping
speed above 80 for shape 892_2_9, dropping
speed above 80 for shape 928_0_58, dropping
negative speed for shape 928_0_58, dropping
negative speed for shape 972_8_13, dropping
negative speed for shape 972_8_14, dropping
negative speed for shape 705_3_34, dropping
negative speed for shape 894_3_31, dropping
negative speed for shape 834_8_20, dropping
speed above 80 for shape 510_1_326, dropping
negative speed for shape 4_2_49, dropping
negative speed for shape 985_9_8, dropping
negative speed for shape 985_9_10, dropping
speed above 80 for shape 530_2_300, dropping
negative speed for shape 894_2_27, dropping
negative speed for shape 894_2_29, dropping
negative speed for shape 4_3_54, dropping
negative speed for shape 967_9_51, dropping
negative speed for shape

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:634: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:572: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [79]:
# m

In [66]:
# m.save(f'./tripmaps/{mts_itp_id}_trolley_02_17_pm_peak.html')

In [90]:
mts._filter(mts.rt_trips).mean_speed_mph.median()

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}


11.927755307468423

In [98]:
mts._filter(mts.rt_trips) >> filter(_.route_id == '7')

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': None, 'direction_id': None, 'direction': None}


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,median_time,direction,mean_speed_mph
1127,278,0,2022-02-17,1880638916631944170,16256687,7,0,7_0_270,2022-01-30,2099-01-01,18:59:28.500000,Eastbound,10.417733
1128,278,0,2022-02-17,1824426740565598728,16256806,7,0,7_0_270,2022-01-30,2099-01-01,18:23:21,Eastbound,8.534365
1129,278,0,2022-02-17,1935353467091061239,16256700,7,0,7_0_270,2022-01-30,2099-01-01,15:02:47,Eastbound,7.633717
1130,278,0,2022-02-17,-2992906560718289806,16256662,7,0,7_0_270,2022-01-30,2099-01-01,16:40:31,Eastbound,7.004786
1131,278,0,2022-02-17,-3391981079614711108,16256695,7,0,7_0_270,2022-01-30,2099-01-01,18:54:48,Eastbound,9.819525
1132,278,0,2022-02-17,4938008090968124399,16256659,7,0,7_0_270,2022-01-30,2099-01-01,15:22:04,Eastbound,9.742212
1133,278,0,2022-02-17,-7700853119818832010,16256764,7,0,7_0_270,2022-01-30,2099-01-01,15:40:04,Eastbound,7.955607
1134,278,0,2022-02-17,3330979155183212700,16256702,7,0,7_0_270,2022-01-30,2099-01-01,17:23:51,Eastbound,8.193739
1135,278,0,2022-02-17,7844536789425594326,16256727,7,0,7_0_270,2022-01-30,2099-01-01,18:32:32,Eastbound,8.816805
1136,278,0,2022-02-17,-7498561325857860072,16256744,7,0,7_0_270,2022-01-30,2099-01-01,16:47:57,Eastbound,7.989574


In [99]:
m = mts.position_interpolators['16256662']['rt'].detailed_speed_map()

/home/jovyan/data-analyses/rt_delay/rt_analysis.py:109: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [101]:
# m

In [102]:
m.save(f'./tripmaps/{mts_itp_id}_rt_07_tr_16256662.html')

## NCTD

In [35]:
analysis_date = dt.date(2022, 2, 8) ##tuesday, new tables

In [36]:
nctd_itp_id = 226

In [37]:
nctd = rt.OperatorDayAnalysis(nctd_itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found_parquet
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
could not generate delays for trip 15765980-NC2110-NCTD-Weekday-20
time data '24:05:00' does not match format '%H:%M:%S'


In [68]:
nctd.set_filter(start_time='15:00', end_time='19:00', route_ids=nc_rail_all)

{'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': ['398', '498', '399'], 'direction_id': None, 'direction': None}


In [69]:
m = nctd.segment_speed_map()

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': ['398', '498', '399'], 'direction_id': None, 'direction': None}


Input geom 1TopologyException: Input geom 1 is invalid: Self-intersection at 245611.20865780726 -533434.45357998728
 is INVALID: Self-intersection at or near point 245611.20865780726 -533434.45357998728 (245611.20865780726308 -533434.45357998728286)
<A>
MULTIPOLYGON (((246950.7672806830669288 -532005.3026747559197247, 246897.2573147887014784 -532014.1165019599720836, 246928.4262990281276871 -531969.7375156998168677, 246950.7672806830669288 -532005.3026747559197247)), ((245615.1499025252996944 -533399.9907900973921642, 245601.4390231675060932 -533435.4231267983559519, 245608.8507208010123577 -533455.0717078715097159, 245615.1499025252996944 -533399.9907900973921642)), ((245644.2556810270762071 -533431.1739654304692522, 245601.4390231675060932 -533435.4231267983559519, 245608.5669568150769919 -533454.3194430114235729, 245644.2556810270762071 -533431.1739654304692522)))
</A>
/home/jovyan/data-analyses/rt_delay/rt_analysis.py:572: UserWarning: Geometry is in a geographic CRS. Results from 

In [57]:
# m

In [56]:
nctd._filter(nctd.rt_trips).mean_speed_mph.median()

view filter: {'start_time': datetime.time(15, 0), 'end_time': datetime.time(19, 0), 'route_ids': ['350', '351', '303', '304', '305', '352', '354', '301', '315', '302', '356', '332', '359', '355', '309', '353', '347', '318', '323', '306', '308', '444', '313', '445', '334', '388', '325', '358', '311', '357'], 'direction_id': None, 'direction': None}


13.768394241852054

In [52]:
coaster = ['398', '498']
sprinter = ['399']
nc_rail_all = coaster + sprinter
nc_bus = [route for route in list(
                   nctd.rt_trips.route_id.unique()) if route not in nc_rail_all
                         ]

In [83]:
# m.save(f'./tripmaps/{nctd_itp_id}_rail_02_17_pm_peak.html')